In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
pd.set_option('display.max_colwidth', None)
from scipy.spatial.distance import cdist
import warnings
warnings.filterwarnings('ignore')

In [2]:
import accelerate
import bitsandbytes

In [3]:
data = pd.read_csv('data.csv')

In [4]:
data.head()

,Genre,Lyric_cleaned,target_cleaned
0,Metal,Let there be lions\nLet them free me from my skin\nLet there be water\nI wanna drown and touch the ground\nAs I drink the water of this earth\nAs I breathe the air in front of meMISSINGSeek and you may find\nReason to hide\nAmong my own kind\nLet me be\nIm falling towards the sun,Live in fear\nLive in fear
1,Metal,Live in fear\nLive in fearMISSINGThis here is my solitude\nIt stays with me\nMy source of fortitude,Seek and you may find\nReason to hide\nAmong my own kind\nLet me be\nIm falling towards the sun
2,Metal,Seek and you may find\nReason to hide\nAmong my own kind\nLet me be\nIm falling towards the sunMISSINGI have seen through the Hyenas eyes,This here is my solitude\nIt stays with me\nMy source of fortitude
3,Metal,This here is my solitude\nIt stays with me\nMy source of fortitudeMISSINGHigh I wanna fly\nTo spread my wings fall from the sky\nDeep I wanna dive\nTo give in to the depths to feel alive\nLike a storm Im comin over you\nLike a wave take you away,I have seen through the Hyenas eyes
4,Metal,I have seen through the Hyenas eyesMISSINGLive in fear\nLive in fear,High I wanna fly\nTo spread my wings fall from the sky\nDeep I wanna dive\nTo give in to the depths to feel alive\nLike a storm Im comin over you\nLike a wave take you away


In [5]:
data.dropna(inplace=True)

In [6]:
data.isna().sum()

Genre             0
Lyric_cleaned     0
target_cleaned    0
dtype: int64

In [7]:
data.reset_index(inplace = True)

In [8]:
!pip list

Package                            Version
---------------------------------- -------------------
accelerate                         0.29.2
aiohttp                            3.9.3
aiosignal                          1.3.1
alabaster                          0.7.12
anaconda-client                    1.7.2
anaconda-navigator                 2.0.3
anaconda-project                   0.9.1
anyio                              2.2.0
appdirs                            1.4.4
argh                               0.26.2
argon2-cffi                        20.1.0
asn1crypto                         1.4.0
astroid                            2.5
astropy                            4.2.1
async-generator                    1.10
async-timeout                      4.0.3
atomicwrites                       1.4.0
attrs                              20.3.0
autopep8                           1.5.6
Babel                              2.9.0
backcall                           0.2.0
backports.functools-lru-cache      1.6.

In [9]:
from sklearn.model_selection import train_test_split
train_dataset,test_dataset = train_test_split(data,test_size=0.1, random_state=42)

In [10]:
train_dataset.reset_index(inplace=True)
test_dataset.reset_index(inplace=True)

In [11]:
train_dataset.shape,test_dataset.shape

((179962, 5), (19996, 5))

In [12]:
train_dataset = train_dataset[['Genre', 'Lyric_cleaned', 'target_cleaned']]
test_dataset = test_dataset[['Genre', 'Lyric_cleaned', 'target_cleaned']]

In [13]:
test_dataset.columns

Index(['Genre', 'Lyric_cleaned', 'target_cleaned'], dtype='object')

In [14]:
from datasets import Dataset, DatasetDict, load_dataset

# temp_train_df = keyword_train_df[['Lyric', 'genre', 'keywords']]
# temp_test_df = keyword_test_df[['Lyric', 'genre', 'keywords']]



train_dataset = Dataset.from_pandas(train_dataset)
test_dataset = Dataset.from_pandas(test_dataset)
dataset = DatasetDict({"train": train_dataset, "test": test_dataset})
# dataset = load_dataset("D3STRON/music_lyrics_5k")

import transformers
import bitsandbytes as bnb
from transformers import AutoModelForCausalLM , AutoTokenizer
from transformers import pipeline, set_seed
from transformers import get_linear_schedule_with_warmup, AdamW
from transformers import AutoConfig
from transformers import BitsAndBytesConfig
from trl import SFTTrainer
#from lightning.pytorch.loggers import TensorBoardLogger

#Dataset
from datasets import load_dataset

#PEFT
from peft import LoraConfig
from peft import PeftConfig
from peft import PeftModel, PeftModel
from peft import get_peft_model
from peft import prepare_model_for_kbit_training
import torch

import warnings
warnings.filterwarnings("ignore")

torch.set_float32_matmul_precision('medium')
torch.cuda.empty_cache()




In [15]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Genre', 'Lyric_cleaned', 'target_cleaned'],
        num_rows: 179962
    })
    test: Dataset({
        features: ['Genre', 'Lyric_cleaned', 'target_cleaned'],
        num_rows: 19996
    })
})

In [16]:
from huggingface_hub import notebook_login
notebook_login()

In [17]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"Trainable params: {trainable_params} || All params: {all_param} || Trainable %: {100 * trainable_params / all_param}")


from transformers import AutoTokenizer

# model_name = "microsoft/phi-2"
# tokenizer_name = "microsoft/phi-2"

model_name = "meta-llama/Llama-2-7b-hf"
tokenizer_name = "meta-llama/Llama-2-7b-hf"

# model_name = "D3STRON/multi_genre_music_generator"
# tokenizer_name = "D3STRON/multi_genre_music_generator"

#Bits and Bytes config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, #4bit quantizaition - load_in_4bit is used to load models in 4-bit quantization 
    bnb_4bit_use_double_quant=True, #nested quantization technique for even greater memory efficiency without sacrificing performance. This technique has proven beneficial, especially when fine-tuning large models
    bnb_4bit_quant_type="nf4", #quantization type used is 4 bit Normal Float Quantization- The NF4 data type is designed for weights initialized using a normal distribution
    bnb_4bit_compute_dtype=torch.bfloat16, #modify the data type used during computation. This can result in speed improvements. 
)

model = AutoModelForCausalLM.from_pretrained(model_name,
                                                      device_map="auto",
                                                      trust_remote_code=True, 
                                                      quantization_config=bnb_config
                                            )
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

# config_name = "ShushantLLM/LLAMA_lyric_generator"
# config = PeftConfig.from_pretrained(config_name)
# tokenizer = AutoTokenizer.from_pretrained(config_name)


# for name, param in model.named_parameters():
#   if "lora" in name:  # Adjust the keyword based on your LoRA layer naming convention
#     param.requires_grad = True



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
print_trainable_parameters(model)

Trainable params: 262410240 || All params: 3500412928 || Trainable %: 7.496550989769399


In [19]:
tokenizer.pad_token = tokenizer.eos_token

tokenizer.add_special_tokens({'pad_token': '<PAD>'})

# Enable gradient checkpointing for the model. Gradient checkpointing is a technique used to reduce the memory consumption during the backward pas. Instead of storing all intermediate activations in the forward pass (which is what's typically done to compute gradients in the backward pass), gradient checkpointing stores only a subset of them
model.gradient_checkpointing_enable() 

# Prepare the model for k-bit training . Applies some preprocessing to the model to prepare it for training.
model = prepare_model_for_kbit_training(model)


# If targeting all linear layers
target_modules = ['q_proj','k_proj','v_proj','o_proj','gate_proj','down_proj','up_proj','lm_head']

    
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules = target_modules,
    bias="none",
    lora_dropout=0.1,
    task_type="CAUSAL_LM",
)

model.add_adapter(lora_config)


# Print the number of trainable parameters in the model
print_trainable_parameters(model)




Trainable params: 40554496 || All params: 3540967424 || Trainable %: 1.1452942414869276


In [23]:
from transformers import TrainingArguments

#Training Parameters
batch_size = 4
output_dir = f"LLama_missing_music_generator"
per_device_train_batch_size = batch_size
gradient_accumulation_steps = 4
optim = 'adamw_hf' #"paged_adamw_32bit" #"paged_adamw_8bit"
save_steps = 10
save_total_limit=3
logging_steps = 10
learning_rate = 1e-5
max_grad_norm = 0.3
warmup_ratio = 0.04
lr_scheduler_type = 'constant_with_warmup'#"cosine"
epochs=1

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    num_train_epochs=epochs, 
    save_steps=save_steps,
    save_total_limit=save_total_limit,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    save_strategy='steps',
    max_grad_norm=max_grad_norm,
    # max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,
    push_to_hub=True,
)

In [24]:
def formatting_func(example):
    text = f"### USER: Generate missing lyrics of : {example['Lyric_cleaned']} \n### ASSISTANT: {example['target_cleaned']}"
    return text


trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=dataset["train"],
    packing=True,
    #dataset_text_field="id",
    tokenizer=tokenizer,
    max_seq_length=2048,
    formatting_func=formatting_func,
)

Generating train split: 0 examples [00:00, ? examples/s]

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [25]:
trainer.train()

Step,Training Loss
10,2.757300
20,2.746200
30,2.760300
40,2.669800
50,2.610900
60,2.563100
70,2.470100
80,2.468500
90,2.425000
100,2.418800


TrainOutput(global_step=1240, training_loss=2.28213635260059, metrics={'train_runtime': 19794.6051, 'train_samples_per_second': 1.003, 'train_steps_per_second': 0.063, 'total_flos': 1.620717140485079e+18, 'train_loss': 2.28213635260059, 'epoch': 1.0})

In [26]:
trainer.push_to_hub(commit_message="Training complete", tags="missing lyric Llama2 Large")

CommitInfo(commit_url='https://huggingface.co/ShushantLLM/LLama_missing_music_generator/commit/8ee21909237e281fa8cbead64e5016dea485f871', commit_message='Training complete', commit_description='', oid='8ee21909237e281fa8cbead64e5016dea485f871', pr_url=None, pr_revision=None, pr_num=None)

In [27]:
trainer.save_model()

In [32]:
Lyric = "Let there be lions\nLet them free me from my skin\nLet there be water\nI wanna drown and touch the ground\nAs I drink the water of this earth\nAs I breathe the air in front of meMISSINGSeek and you may find\nReason to hide\nAmong my own kind\nLet me be\nIm falling towards the sun"
prompt = f"""### USER: Generate missing lyrics of : {Lyric} \n### ASSISTANT:""".strip()
print(prompt)

### USER: Generate missing lyrics of : Let there be lions
Let them free me from my skin
Let there be water
I wanna drown and touch the ground
As I drink the water of this earth
As I breathe the air in front of meMISSINGSeek and you may find
Reason to hide
Among my own kind
Let me be
Im falling towards the sun 
### ASSISTANT:


In [29]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [30]:
import numpy as np
# from tqdm.notebook import tqdm

def choose_from_top(probs, cur_ids, n=7, no_rep=2
):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob /= np.sum(top_prob)  # Normalize

    cur_ids_list = cur_ids.squeeze().cpu().numpy().tolist()
    for i in range(100):
        if i <= 5:
            choice = np.random.choice(ind, 1)[0]
        else:
            choice = np.random.choice(ind, 1, p=top_prob)[0]
        n_gram = cur_ids_list[-no_rep + 1 :] + [choice]
        if any(cur_ids_list[i : i + no_rep] == n_gram for i in range(len(cur_ids_list) - no_rep + 1)):
            continue
        return choice
    return np.random.choice(ind, 1, p=top_prob)[0]

def generate_response(Lyric : str) -> str:
    prompt =  f"""### ### USER: Generate missing lyrics of : {Lyric} \n### ASSISTANT:""".strip()
    cur_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    with torch.no_grad():
        for i in range(50):
            outputs = model(cur_ids)
            logits = outputs['logits']
            softmax_logits = torch.softmax(logits[0,-1], dim=0) #Take the first(from only one in this case) batch and the last predicted embedding
            if i == 5:
                n = 30
            else:
                n = 3
            next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), cur_ids, n=n) #Randomly(from the topN probability distribution) select the next word
            cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim = 1) # Add the last word to the running sequence

            if next_token_id in tokenizer.encode('<|endoflyric|>'):
                joke_finished = True
                break
            output_list = list(cur_ids.squeeze().to('cpu').numpy())
            output_text = tokenizer.decode(output_list)
#             print(output_text)
    print(output_text)

In [33]:
print(generate_response(Lyric))

<s> ### ### USER: Generate missing lyrics of : Let there be lions
Let them free me from my skin
Let there be water
I wanna drown and touch the ground
As I drink the water of this earth
As I breathe the air in front of meMISSINGSeek and you may find
Reason to hide
Among my own kind
Let me be
Im falling towards the sun 
### ASSISTANT: Im not a man
Who isnt scared to fall
But if Im gon to die I need to feel it all around me</s>sit down and listen to the story
sitting in a chair in the corner in the darkest
None
